### PreTangPoetry explorer

In [1]:
# import PTPoetry class (derived from the Book class)
from PTPoetry import PTPoetry
from bs4 import BeautifulSoup
import bs4
import re, sys

In [2]:
MyBooks = PTPoetry(creator="RGT", date="2018-11-19")

In [3]:
%%time
MyBooks.load_htmls(path="../Four Hanji texts/PreTangPoetry/", limit=2000)

100%|████████████████████████████████████████████████████████████████████████| 2000/2000 [00:03<00:00, 588.42it/s]


Wall time: 3.47 s


In [4]:
# Randomly checks a bookmark
idx = 701 #1451
MyBooks.flat_bodies[idx].body.a.text

'集／總集／先秦漢魏晉南北朝詩／漢詩／卷十／樂府古辭二／雜曲歌辭／古歌(P.297)'

In [5]:
%%time
MyBooks.strip_all_irrelevant_tags()

100%|████████████████████████████████████████████████████████████████████████| 2000/2000 [00:03<00:00, 580.94it/s]
INFO:root:Remove 標註, page number, and page dividers from the tree structure.
100%|████████████████████████████████████████████████████████████████████████| 2000/2000 [00:02<00:00, 959.50it/s]
INFO:root:Remove the new lines added by the page dividers, connect the paragraphs before and after the new lines.


Wall time: 6.8 s


### Preliminary exploration
##### We want to find out how many nodes (children) there are under each flat_bodies[i]

In [6]:
%%time
# This block asserts that each of the flat_bodies nodes has exactly 5 children and
# that the children at index positions 0, 2, 4 are all newlines ('\n') 
cnt = 0
for bs in MyBooks.flat_bodies:
    assert len(list(bs.html.body.children)) == 5
    assert list(bs.html.body.children)[0] == '\n'
    assert list(bs.html.body.children)[2] == '\n'
    assert list(bs.html.body.children)[4] == '\n'
    cnt += 1
print(cnt)

2000
Wall time: 290 ms


### Now extract paths

In [7]:
%%time
MyBooks.extract_paths()

100%|██████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 14424.67it/s]


Wall time: 144 ms


In [18]:
# check some random page
idx = 1072
MyBooks.paths[idx]

'集／總集／先秦漢魏晉南北朝詩／魏詩／卷十／阮籍／詠懷詩八十二首(P.496)'

In [19]:
MyBooks.title_candidates[idx]

'詠懷詩八十二首'

### Next we explore the children of html.body.span

In [ ]:
%%time
# This block of code finds out how many children each of the main (<span>) nodes
# has and what type(s) of nodes its children are
cnt = 0
counts = {}
for i, bs in enumerate(MyBooks.flat_bodies):
    n = len(list(bs.html.body.span))
    if n in counts:
        counts[n].append(i)
    else:
        counts[n] = [i]
    cnt += 1
print(cnt)

In [21]:
counts.keys()

dict_keys([])

In [ ]:
# what are the indexes that contain 'cnt' nodes
cnt = 9
', '.join([str(i) for i in counts[cnt]])

In [14]:
# clean children of html.body.span (strips empty nodes)
MyBooks.clean_children()

100%|███████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 7353.16it/s]


In [23]:
idx

1072

In [24]:
#idx = 694
titles = []
poem_texts = []
for c in MyBooks.flat_bodies[idx]:
    print(c.name)
    pass
    if c.name in ('h3', 'b'):
        titles.append(c.text)
    elif c.name == 'div':
        poem_texts.append(c.text)
poem_texts
MyBooks.flat_bodies[idx]
titles

b
div


['\u3000\u3000詠懷詩八十二首']

In [27]:
MyBooks.title_candidates[idx]
MyBooks.paths[idx]

'集／總集／先秦漢魏晉南北朝詩／魏詩／卷十／阮籍／詠懷詩八十二首(P.496)'

In [28]:
MyBooks.flat_bodies[idx]

<span id="fontstyle" style="FONT-SIZE: 12pt;letter-spacing:1pt; LINE-HEIGHT: 18pt;width:99%;word-break:break-all"><b><h3>　　詠懷詩八十二首</h3></b><div style="text-indent:0em;padding-left:3em;"> <font size="-2">詩紀云。阮嗣宗集傳之既久。頗存偽闕。世之較錄者往往肆為補綴。作者之旨。淆亂甚焉。今以諸本參校。其義稍優者為正文。互異者分註於下。其舊有闕文疑字而今本竄益者。廓其傍。俟再考正。</font> 夜中<span id="q00" style="display:none;width:;height:;color:green;font-size:13px">秘府論作中夜。</span>不能寐。<span id="q01" style="display:none;width:;height:;color:green;font-size:13px">初學記或作寢。</span>起坐彈鳴琴。薄帷鑒<span id="q02" style="display:none;width:;height:;color:green;font-size:13px">李善本文選作鑑。同。</span>明月。清風吹我襟。<span id="q03" style="display:none;width:;height:;color:green;font-size:13px">文選作衿。類聚、初學記、御覽、文章正宗同。</span>孤鴻號外野。翔<span id="q04" style="display:none;width:;height:;color:green;font-size:13px">李善本文選作朔。六臣本注云。善作朔字。文章正宗作朔。</span>鳥鳴<span id="q05" style="display:none;width:;height:;color:green;font-size:13px">類聚作歸。</span>北林。徘徊將何見。憂思獨傷心。<span id="q06" style="display:none;width:;height:;color:green;font-s

In [101]:
def extract_collation_notes(bs):
    ''' bs: can be an h3, div or other types of tags 
        returns: a dictionary of collation notes keyed with qxx 
    '''
    collation_notes = {}
    for t in bs.find_all('span', {'id': re.compile(pattern=r'^q\d+')}):
        a = t.get_attribute_list('id', )[0]  # ['q00']
        collation_notes[a] = t.text
    return collation_notes
    

In [86]:
def extract_title_poem_collation_notes(bs):
    ''' title: Title (in h3), possibly with collcation notes
        poem:  text of poem, possibly with collcation notes
        collcation_notes: a dictionary of collcation notes with <span> id (q00, q01, etc.) as keys
    '''
    cnt = 0
    rb = bs.html.body
    raw_body_text = str(rb)
    collation_notes = []
    for t in rb.find_all('span', {'id': re.compile(pattern=r'^q\d+')}):
        cnt += 1
        collation_notes.append(t.text)
        raw_body_text = raw_body_text.replace(str(t), "{}")

    rb = BeautifulSoup(raw_body_text, 'lxml') # parse it again
    raw_body = []
    for d in rb.html.body.find_all('div'):
        raw_body.append(d.text)
    return (raw_body, collation_notes)

In [88]:
rb, cn = extract_poem_plus_collation_notes(bs)

In [67]:
len(cn), rb[0].count("{}")

(22, 21)

### Notes
##### (1) For <span id='qxxx'></span>, the text for q00 or q000 (etc.) is the first collocation note, whether it is attached to the Title or not

In [8]:
print(chr(0x470E)) # the rare character 䜎

䜎


In [ ]:
tag = PTPoetry.flat_bodies[idx]    # 1072,  1451
cnt = 0
#for t in tag.find_all('span', {'id': {"q00", "q01", "q02"}}):
notes_id = []
for t in tag.find_all('span'):
    cnt += 1
    attr_list = t.get_attribute_list('id')
    if attr_list[0].startswith("q"):  # only id='qxxx', where xxx is a number, is picked up
        notes_id.extend(attr_list)

cnt = 0
for t in tag.find_all('span', {'id': set(notes_id)}):
    cnt += 1
    #print(cnt, ":", t.text)

raw_body = str(tag.html.body)
type(raw_body)
for j, note_id in enumerate(notes_id):
    x = tag.find('span', id=note_id)
    raw_body = raw_body.replace(str(x), "{}")
    print(j, note_id, x.text)
raw_body

In [12]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_body, 'lxml')
opening_note = soup.find("div").find_all("font", size="-2")
if opening_note:
    print(opening_note[0].text)

詩紀云。阮嗣宗集傳之既久。頗存偽闕。世之較錄者往往肆為補綴。作者之旨。淆亂甚焉。今以諸本參校。其義稍優者為正文。互異者分註於下。其舊有闕文疑字而今本竄益者。廓其傍。俟再考正。


In [33]:
if opening_note:
    poem_body = soup.find("div").text.replace(opening_note[0].text, "").strip()
poem_body.count("{}")

270

In [ ]:
tag = PTPoetry.flat_bodies[1451]
raw_text = tag.html.body
str(raw_text)

In [25]:
raw_body.count("{}"), len(notes_id)
#assert raw_body.count("{}") == len(notes_id)
#notes_id
#tag.find('span', id=notes_id[-1]).text

(270, 270)

In [26]:
raw_body.count("{}")

270

In [5]:
# Pulls out the bookmark
PTPoetry.extract_paths()

In [ ]:
[(idx, value) for idx, value in enumerate(PTPoetry.paths[1500:1520])]

In [10]:
SplitPaths = []
MetaList = []
MetaList2 =[]

for path in PTPoetry.paths:
    x = path.split("／")
    SplitPaths.append(x)

for path2 in SplitPaths:
    try:
        #making our collection this a list, not a tuple
        y = [path2[3], path2[4], path2[5], path2[-1]]
        MetaList.append(y)
    except:
        print(path2)
        pass

#But the method above is not right.  Authors not always at index 5, and at the start
#of each author, index-1 will give the author, not the title.

for item in MetaList:
    titlepage = item[3].strip(")").split("(")
    if len(titlepage) == 1:
        pass
    MetaList2.append(item[0:3] + titlepage)
    #TODO: failing to strip out the "P." in the page number

['集', '總集', '先秦漢魏晉南北朝詩', '梁詩', '卷二十三(P.1980)']
['集', '總集', '先秦漢魏晉南北朝詩', '梁詩', '卷二十四(P.2008)']
['集', '總集', '先秦漢魏晉南北朝詩', '梁詩', '卷二十五(P.2030)']
['集', '總集', '先秦漢魏晉南北朝詩', '梁詩', '卷二十六(P.2061)']
['集', '總集', '先秦漢魏晉南北朝詩', '梁詩', '卷二十七(P.2079)']
['集', '總集', '先秦漢魏晉南北朝詩', '梁詩', '卷二十八(P.2107)']
['集', '總集', '先秦漢魏晉南北朝詩', '梁詩', '卷二十九(P.2136)']
['集', '總集', '先秦漢魏晉南北朝詩', '梁詩', '卷三十(P.2162)']
['集', '總集', '先秦漢魏晉南北朝詩', '北魏詩(P.2195)']
['集', '總集', '先秦漢魏晉南北朝詩', '北魏詩', '卷一(P.2195)']
['集', '總集', '先秦漢魏晉南北朝詩', '北魏詩', '卷二(P.2212)']
['集', '總集', '先秦漢魏晉南北朝詩', '北魏詩', '卷三(P.2227)']
['集', '總集', '先秦漢魏晉南北朝詩', '北魏詩', '卷四(P.2246)']
['集', '總集', '先秦漢魏晉南北朝詩', '北齊詩(P.2255)']
['集', '總集', '先秦漢魏晉南北朝詩', '北齊詩', '卷一(P.2255)']
['集', '總集', '先秦漢魏晉南北朝詩', '北齊詩', '卷二(P.2272)']
['集', '總集', '先秦漢魏晉南北朝詩', '北齊詩', '卷三(P.2287)']
['集', '總集', '先秦漢魏晉南北朝詩', '北齊詩', '卷四(P.2301)']
['集', '總集', '先秦漢魏晉南北朝詩', '北周詩(P.2322)']
['集', '總集', '先秦漢魏晉南北朝詩', '北周詩', '卷一(P.2322)']
['集', '總集', '先秦漢魏晉南北朝詩', '北周詩', '卷二(P.2345)']
['集', '總集', '先秦漢魏晉南北朝詩', '北周詩', '卷三(P.2365)']

In [11]:
print(MetaList[100], '\n', MetaList2[100])

['先秦詩', '卷三', '謠', '包山謠(P.43)'] 
 ['先秦詩', '卷三', '謠', '包山謠', 'P.43']


In [12]:
#let's get a rawer set of bookmark data, to work with.  Everything except
# the first three spots (集，總集，先秦漢魏晉南北朝詩)
RoughMeta = []
for path in PTPoetry.paths:
    x = path.split("／")
    RoughMeta.append(x[3:])


In [13]:
RoughMeta[1300:1350]

[['晉詩', '卷二', '棗據', '詩(P.589)'],
 ['晉詩', '卷二', '棗據', '五言詩(P.590)'],
 ['晉詩', '卷二', '棗據', '詩(P.590)'],
 ['晉詩', '卷二', '王濬(P.591)'],
 ['晉詩', '卷二', '王濬', '祖道應令詩(P.591)'],
 ['晉詩', '卷二', '王濬', '平吴詩(P.591)'],
 ['晉詩', '卷二', '荀勗(P.591)'],
 ['晉詩', '卷二', '荀勗', '從武帝華林園宴詩(P.592)'],
 ['晉詩', '卷二', '荀勗', '三月三日從華林園詩(P.592)'],
 ['晉詩', '卷二', '王銓(P.592)'],
 ['晉詩', '卷二', '王銓', '為兩足虎作歌詩(P.592)'],
 ['晉詩', '卷二', '夏侯湛(P.593)'],
 ['晉詩', '卷二', '夏侯湛', '周詩(P.593)'],
 ['晉詩', '卷二', '夏侯湛', '山路吟(P.594)'],
 ['晉詩', '卷二', '夏侯湛', '江上泛歌(P.594)'],
 ['晉詩', '卷二', '夏侯湛', '離親詠(P.594)'],
 ['晉詩', '卷二', '夏侯湛', '長夜謠(P.595)'],
 ['晉詩', '卷二', '夏侯湛', '寒苦謠(P.595)'],
 ['晉詩', '卷二', '夏侯湛', '春可樂(P.595)'],
 ['晉詩', '卷二', '夏侯湛', '秋可哀(P.596)'],
 ['晉詩', '卷二', '夏侯湛', '秋夕哀(P.596)'],
 ['晉詩', '卷二', '夏侯湛', '征邁辭(P.596)'],
 ['晉詩', '卷二', '王濟(P.597)'],
 ['晉詩', '卷二', '王濟', '平吴後三月三日華林園詩(P.597)'],
 ['晉詩', '卷二', '王濟', '從事華林詩(P.597)'],
 ['晉詩', '卷二', '王濟', '詩(P.597)'],
 ['晉詩', '卷二', '王濟', '答何劭詩(P.598)'],
 ['晉詩', '卷二', '孫楚(P.598)'],
 ['晉詩', '卷二', '孫楚', '答弘農故吏民詩(

In [14]:
import pandas as pd
RoughFrame=pd.DataFrame(RoughMeta)

In [15]:
RoughFrame[900:1100]
#from this we can hypothesize: if index-3 is "None", then that item is an author listing.
# We could exclude those.

,0,1,2,3,4,5
900,魏詩,卷五,吴質(P.412),None,None,None
901,魏詩,卷五,吴質,思慕詩(P.412),None,None
902,魏詩,卷五,麋元(P.412),None,None,None
903,魏詩,卷五,麋元,詩(P.412),None,None
904,魏詩,卷五,魏明帝曹叡(P.413),None,None,None
905,魏詩,卷五,魏明帝曹叡,樂府,短歌行(P.413),None
906,魏詩,卷五,魏明帝曹叡,樂府,善哉行(P.413),None
907,魏詩,卷五,魏明帝曹叡,樂府,同前四解(P.414),None
908,魏詩,卷五,魏明帝曹叡,樂府,步出夏門行(P.414),None
909,魏詩,卷五,魏明帝曹叡,樂府,月重輪行(P.415),None


In [16]:
#a simple function to return list of works by an author.
def findauthorworks(author=""):
    for item in MetaList2:
        if author in item[2]:
            print(item)

In [ ]:
findauthorworks(author="曹植")

In [18]:
#Count total number of authors (using `set` type)
#TODO: discriminate between authors and labels

AuthorList=[]
for item in MetaList2:
    AuthorList.append(item[2])

AuthorSet = set(AuthorList)
print("Total discrete authors/labels: {}".format(len(AuthorSet)))

Total discrete authors/labels: 1545


In [ ]:
#using a counter to get the indexes
counter = -1
for item in MetaList2:
    counter +=1
    if "附" in item[2]:
        print(counter, item)

In [ ]:
PTPoetry.flat_bodies[1148:1159]

## OK, the metadata will take some more organization.  We want to look at the texts.

In [ ]:
# trying to adapt a method from the `Song shu` subclass, but that is confusing...
# first have a look:
PTPoetry.flat_bodies[10]

In [22]:
TitlePoem = []
for poem in PTPoetry.flat_bodies:
    title = poem.find('h3')
    text = poem.find('div', attrs={'style': "text-indent:0em;padding-left:0em;"})
    TitlePoem.append((title, text))
#    PTPoetry.flat_passages.append((title, text))
    

Problems:

- flat_passages doesn't fill in correctly.
- TitlePoem list does, but my method is including the notes, and I don't want that.

:point_right: My BS skills are not up to snuff

In [120]:
TitlePoem[203][1]

<div style="text-indent:0em;padding-left:0em;">力拔山兮氣蓋世。時不利兮騅不逝。騅不逝兮可奈何。虞兮虞兮奈若何。<a href="#" onclick="q00.style.display=q00.style.display=='none'?'':'none'" title="夾註"><img align="absmiddle" border="0" src="/ihp/qn.gif"/></a><span id="q00" style="display:none;width:;height:;color:green;font-size:13px">○漢書項羽傳。史記項羽本紀。御覽八十七、五百七十。樂府詩集五十八作力拔山操。文選補遺三十五作垓下帳中歌。詩紀二作垓下歌。○逯案。文選補遺與詩紀標題各異。然皆涉杜撰。今從漢書只曰歌。樂府詩集引琴集云。力拔山操。項羽所作也。則此歌曰操。亦後起之名。</span> </div>

## failed tries to strip down to titles and texts...

In [126]:
PTPoetry.strip_all_irrelevant_tags()

INFO:root:Remove 標註, page number, and page dividers from the tree structure.
INFO:root:Remove the new lines added by the page dividers, connect the paragraphs before and after the new lines.


In [137]:
PTPoetry.strip_tag('span', {'id':'q00'})

In [77]:
#adapting from `Song shu` subclass

for body,path in zip(PTPoetry.flat_bodies, PTPoetry.paths):
    texts  = body.find_all('div', attrs={'style': "text-indent:0em;padding-left:0em;"})
    PTPoetry.flat_passages.append(texts)

In [78]:
PTPoetry.flat_passages[10]

[<div style="text-indent:0em;padding-left:3em;"> <font size="-2">史記曰。箕子朝周。過故殷墟。感宮室毀壞生禾黍。箕子傷之。欲哭則不可。欲泣為其近婦人。乃作麥秀之詩以歌之。其詩曰。</font> </div>,
 <div style="text-indent:0em;padding-left:0em;">麥秀漸漸<a href="#" onclick="q00.style.display=q00.style.display=='none'?'':'none'" title="夾註"><img align="absmiddle" border="0" src="/ihp/qn.gif"/></a><span id="q00" style="display:none;width:;height:;color:green;font-size:13px">文選注不重漸字。尚書大傳作蔪蔪。</span>兮禾黍油油。<a href="#" onclick="q01.style.display=q01.style.display=='none'?'':'none'" title="夾註"><img align="absmiddle" border="0" src="/ihp/qn.gif"/></a><span id="q01" style="display:none;width:;height:;color:green;font-size:13px">文選注作黍米目黽目黽。</span>彼狡童<a href="#" onclick="q02.style.display=q02.style.display=='none'?'':'none'" title="夾註"><img align="absmiddle" border="0" src="/ihp/qn.gif"/></a><span id="q02" style="display:none;width:;height:;color:green;font-size:13px">史記作僮。文選注同。</span>兮不與我好兮。<a href="#" onclick="q03.style.display=q03.style.display=='none'?'':'

In [77]:
PTPoetry

       type       variable                 method current_length
0      meta      flat_meta      self.extract_meta              0
1      path          paths     self.extract_paths           7767
2  passages  flat_passages  self.extract_passages              0